In [31]:
#!pip install evidently
#!pip install shap
#!pip install evidently --upgrade
# !pip install dvc

In [4]:
import pandas as pd
import numpy as np
import pickle
import shap
import joblib

from zipfile import ZipFile
import time

import shap
import joblib

# from evidently.dashboard import Dashboard 
# import evidenyly.tabs
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, ClassificationPreset


In [5]:
# import warnings
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

In [6]:
model_drift = pickle.load(open("./models/LGBMClassifier.pkl", "rb"))

In [7]:
z_train = ZipFile("./X_data.zip")
data_train = pd.read_csv(z_train.open('X_data.csv'), encoding ='utf-8') #, index_col='SK_ID_CURR'
#data_train.drop('SK_ID_CURR', axis=1, inplace=True)
data_train.shape

(307511, 120)

In [8]:
data_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Single / not married,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Transport: type 3
0,100002,0.0,0.0,0.001512,0.090287,0.090032,0.077441,0.256321,0.111161,0.964437,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,100003,0.0,0.0,0.002089,0.311736,0.132924,0.271605,0.045016,0.522886,0.933676,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,1.0,1.0,0.000358,0.022472,0.020025,0.023569,0.134897,0.651466,0.987439,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,0.0,0.000935,0.066837,0.109477,0.063973,0.107023,0.649154,0.830337,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,100007,0.0,0.0,0.000819,0.116854,0.078975,0.117845,0.392880,0.701409,0.830393,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
cols = data_train.columns.tolist()
cols

['SK_ID_CURR',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_WORK_PHONE',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_CITY_NOT_LIVE_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'COMMONAREA_MEDI',
 'LIVINGAREA_MEDI',
 'TOTALAREA_MODE',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAY

In [10]:
z_test = ZipFile("./data_test.zip")
data_test = pd.read_csv(z_test.open('data_test.csv'), encoding ='utf-8') #, index_col='SK_ID_CURR'
#data_train.drop('SK_ID_CURR', axis=1, inplace=True)
data_test.shape

(48744, 310)

In [11]:
data_test = data_test[cols]
data_test.shape

(48744, 120)

In [12]:
# data_test.reset_index(inplace = True)
data_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Single / not married,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Transport: type 3
0,100001,0.0,0.0,0.024654,0.238037,0.102453,0.184049,0.257380,0.333427,0.866682,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,0.0,0.0,0.016440,0.080785,0.084558,0.061350,0.491855,0.399339,0.744130,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,100013,0.0,1.0,0.040054,0.280965,0.378515,0.265849,0.260854,0.288794,0.744760,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,100028,0.0,0.0,0.065721,0.695297,0.262078,0.695297,0.361760,0.628269,0.893197,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,100038,0.0,1.0,0.034920,0.263804,0.166995,0.263804,0.135340,0.680685,0.874585,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [13]:
y_train = pd.read_csv('TARGET.csv')
y_train.set_index('SK_ID_CURR', inplace=True)
y_train.shape

(307511, 1)

In [14]:
y_train.head()

,TARGET
SK_ID_CURR,
100002,1.0
100003,0.0
100004,0.0
100006,0.0
100007,0.0


In [20]:
merged_df = data_train.merge(y_train, left_index=True, right_index=True)
merged_df

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_FAMILY_STATUS_Single / not married,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Transport: type 3,TARGET
100002,216093,0.0,0.0,0.000742,0.061519,0.085311,0.057239,0.128209,0.168771,0.899341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100003,216094,0.0,0.0,0.000742,0.197348,0.150439,0.158249,0.109945,0.566967,0.455337,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004,216095,0.0,1.0,0.000742,0.179840,0.120147,0.160494,0.421848,0.749775,0.907995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100006,216098,0.0,0.0,0.001204,0.213483,0.096841,0.214366,0.282797,0.322717,0.659223,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100007,216099,0.0,1.0,0.000700,0.063483,0.025781,0.046016,0.491595,0.811556,0.738332,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307505,456249,0.0,0.0,0.000742,0.044944,0.079695,0.046016,0.311695,0.952368,0.907995,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307506,456251,0.0,0.0,0.001127,0.052360,0.101176,0.046016,0.446855,0.103608,0.986824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307507,456252,0.0,0.0,0.000396,0.056067,0.040505,0.046016,0.344429,0.748929,0.907995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307508,456253,0.0,0.0,0.001089,0.157969,0.110618,0.135802,0.065247,0.421477,0.557782,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df_streamlit = merged_df.sample(50000)
print(df_streamlit.shape)
df_streamlit

(50000, 121)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_FAMILY_STATUS_Single / not married,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Transport: type 3,TARGET
232260,369021,0.0,0.0,0.002589,0.367145,0.150105,0.327722,0.260475,0.830383,0.907995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
272662,416087,0.0,0.0,0.001512,0.072198,0.064900,0.057239,0.260475,0.339290,0.987606,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
236328,373731,0.0,1.0,0.002089,0.148584,0.115620,0.102132,1.000000,0.260823,0.952937,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
261754,403010,0.0,1.0,0.001704,0.136719,0.119568,0.102132,0.129705,0.108794,0.896996,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
188275,318279,1.0,0.0,0.000742,0.050562,0.041962,0.051627,0.048852,0.387937,0.928093,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129571,250283,0.0,1.0,0.001127,0.079874,0.062671,0.068462,0.245631,0.800507,0.251898,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
242159,380356,1.0,0.0,0.000396,0.033708,0.028800,0.034792,0.256321,0.016009,0.959078,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
163306,289323,0.0,0.0,0.000935,0.364045,0.148824,0.364759,0.361433,0.888501,0.907995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295484,442339,0.0,0.0,0.001704,0.086013,0.071762,0.059484,0.134897,0.743236,0.907995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [28]:
df_streamlit['AMT_INCOME_TOTAL'].nunique()

754

In [29]:
df_streamlit['AMT_INCOME_TOTAL'].min()

3.84699722631501e-06

In [23]:
# df_streamlit.to_csv('df_streamlit.csv', index=False)

In [24]:
def clean_column_names(df):
    # Remplacer ou supprimer des caractères spéciaux
    df.columns = df.columns.str.replace(':', '_')
    df.columns = df.columns.str.replace('[', '_')
    df.columns = df.columns.str.replace(']', '_')
    df.columns = df.columns.str.replace('{', '_')
    df.columns = df.columns.str.replace('}', '_')
    df.columns = df.columns.str.replace('|', '_')
    # Vous pouvez ajouter d'autres remplacements au besoin
    return df

In [15]:
# reference = merged_df
# current = data_test

# print(reference.shape)
# print(current.shape)

In [16]:
# reference = clean_column_names(reference)
# current = clean_column_names(current)
# merged_df = clean_column_names(merged_df)

In [17]:
# reference_train = merged_df.loc[:246000,:] #80%
# current_train = merged_df.loc[246001:,:] #20%
# reference_train

On va calculer la prédiction de la feature TARGET sur les datasets df_train et df_test afin de voir si le drift éventuel des features sources engendre un drift au niveau de la cible à prédire.

In [18]:
y_predic_proba = model_drift.predict_proba( data_train.drop(columns=["SK_ID_CURR"]) )
data_train["TARGET"] = np.where(y_predic_proba[:,1]>0.16,1,0)

In [19]:
y_predic_proba = model_drift.predict_proba( data_test.drop(columns=["SK_ID_CURR"]) )
data_test["TARGET"] = np.where(y_predic_proba[:,1]>0.16,1,0)

In [20]:
data_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_FAMILY_STATUS_Single / not married,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Transport: type 3,TARGET
0,100001,0.0,0.0,0.024654,0.238037,0.102453,0.184049,0.257380,0.333427,0.866682,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,100005,0.0,0.0,0.016440,0.080785,0.084558,0.061350,0.491855,0.399339,0.744130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,100013,0.0,1.0,0.040054,0.280965,0.378515,0.265849,0.260854,0.288794,0.744760,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,100028,0.0,0.0,0.065721,0.695297,0.262078,0.695297,0.361760,0.628269,0.893197,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
4,100038,0.0,1.0,0.034920,0.263804,0.166995,0.263804,0.135340,0.680685,0.874585,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0.0,0.0,0.021574,0.167035,0.085138,0.102249,0.024760,0.292602,0.704043,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
48740,456222,0.0,0.0,0.029787,0.262401,0.166111,0.204499,0.491855,0.784510,0.934257,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
48741,456223,0.0,1.0,0.040054,0.122699,0.173381,0.122699,0.361760,0.519292,0.826137,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
48742,456224,0.0,0.0,0.045187,0.184049,0.128073,0.184049,0.257380,0.628717,0.843661,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [21]:
numerical_features = data_train.columns.tolist()
numerical_features = [col for col in numerical_features if col not in ['SK_ID_CURR', 'TARGET']]
len(numerical_features)


119

In [22]:
reference = data_train.loc[:, numerical_features + ["TARGET"] ].sample(frac=1, replace=False)
current = data_test.loc[:, numerical_features + ["TARGET"] ].sample(frac=1, replace=False)

In [23]:
report = Report(metrics=[
    DataDriftPreset(), 
])

# report.run(reference_data=reference, current_data=current)

In [24]:
# report.save_html("./reports/report_data_drift.html")